In [1]:
import h2o_mlops_scoring_client
import os

In [2]:
num_cores = 10
h2o_mlops_scoring_client.spark_master = f"local[{num_cores}]"

In [3]:
h2o_mlops_scoring_client.spark_conf_dir = os.path.expanduser("/Users/kishani/h2oapps/Joe MLOps Scoring Test/conf")


In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.expanduser("/Users/kishani/h2oapps/Joe MLOps Scoring Test/kishani-mlops-scoring-test-2461a12de86c.json")

In [5]:
MLOPS_ENDPOINT_URL = "https://model.cloud-qa.h2o.ai/b17dddbd-6d4b-4d83-b975-29466f4a6a24/model/score"

In [6]:
ID_COLUMN = "ID"
SOURCE_DATA = "kishani-mlops-scoring-test.gbm_dataset.test_table"
SINK_LOCATION = "kishani-mlops-scoring-test.gbm_dataset.mlops_scoring_client"

In [7]:
SOURCE_FORMAT = h2o_mlops_scoring_client.Format.BIGQUERY
SINK_FORMAT = h2o_mlops_scoring_client.Format.BIGQUERY

In [8]:
h2o_mlops_scoring_client.WriteMode.OVERWRITE.value

'Overwrite existing files'

In [9]:
SINK_WRITE_MODE = h2o_mlops_scoring_client.WriteMode.OVERWRITE

In [10]:
def preprocess(spark_df):
    return spark_df.repartition(30)

In [11]:
from pyspark.sql.functions import col

def postprocess(spark_df):
    """Required for GBQ."""
    sanitized_column_names = {}
    for c in spark_df.columns:
        if "." in c:
            sanitized_column_names[c] = c.replace(".", "_")
    return spark_df.select(
        [col(f"`{c}`").alias(sanitized_column_names.get(c, c)) for c in spark_df.columns]
    )

In [12]:
h2o_mlops_scoring_client.score_source_sink(
    mlops_endpoint_url=MLOPS_ENDPOINT_URL,
    id_column=ID_COLUMN,
    source_data=SOURCE_DATA,
    source_format=SOURCE_FORMAT,
    sink_location=SINK_LOCATION,
    sink_format=SINK_FORMAT,
    sink_write_mode=SINK_WRITE_MODE,
    preprocess_method=preprocess,
    postprocess_method=postprocess,
    spark_config_overrides={
        "temporaryGcsBucket": "test-bucket-mlops-scoring-client"
    }
) 

23/05/03 23:41:13 INFO h2o_mlops_scoring_client: Starting Spark context
23/05/03 23:41:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/03 23:41:17 INFO h2o_mlops_scoring_client: Connecting to H2O.ai MLOps scorer at 'https://model.cloud-qa.h2o.ai/b17dddbd-6d4b-4d83-b975-29466f4a6a24/model/score'
23/05/03 23:41:21 INFO h2o_mlops_scoring_client: Applying preprocess method
23/05/03 23:41:21 INFO h2o_mlops_scoring_client: Starting scoring from 'kishani-mlops-scoring-test.gbm_dataset.test_table' to 'kishani-mlops-scoring-test.gbm_dataset.mlops_scoring_client'
23/05/03 23:47:35 INFO h2o_mlops_scoring_client: Stopping Spark context


RetryError: HTTPSConnectionPool(host='model.cloud-qa.h2o.ai', port=443): Max retries exceeded with url: /b17dddbd-6d4b-4d83-b975-29466f4a6a24/model/schema (Caused by ResponseError('too many 404 error responses'))